# Landsat Collections

In [1]:
import ee
ee.Initialize()

In [2]:
from geetools import collection, tools, ui

In [3]:
from ipygee import *

## Default values

In [3]:
collection.landsat.DEFAULTS

{1: {'process': 'RAW', 'sensor': 'MSS'},
 2: {'process': 'RAW', 'sensor': 'MSS'},
 3: {'process': 'RAW', 'sensor': 'MSS'},
 4: {'process': 'SR', 'sensor': 'TM'},
 5: {'process': 'SR', 'sensor': 'TM'},
 7: {'process': 'SR', 'sensor': 'ETM'},
 8: {'process': 'SR', 'sensor': 'OLI'}}

This means that the default collection for Landsat 1 is RAW and MSS, for Landsat 5 is SR and TM

# HELPERS

In [13]:
def print_values(image):
    print(tools.image.getValue(image, image.geometry().centroid(1), 30, side='client'))

## Landsat 8 SR

In [9]:
l8sr = collection.Landsat(8, 'SR')

## Information about the collection

In [6]:
collection.info(l8sr)

{'algorithms': {'brdf': <bound method Landsat.brdf of <geetools.collection.landsat.Landsat object at 0x7f39f129fa90>>,
  'harmonize': <bound method Landsat.harmonize of <geetools.collection.landsat.Landsat object at 0x7f39f129fa90>>,
  'rescale': <bound method Landsat.rescaleAll of <geetools.collection.landsat.Landsat object at 0x7f39f129fa90>>},
 'band_names': ['aerosol',
  'blue',
  'green',
  'red',
  'nir',
  'swir',
  'swir2',
  'thermal',
  'thermal2',
  'sr_aerosol',
  'pixel_qa',
  'radsat_qa'],
 'bands': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B10',
  'B11',
  'sr_aerosol',
  'pixel_qa',
  'radsat_qa'],
 'bit_bands': ['sr_aerosol', 'pixel_qa', 'radsat_qa'],
 'classification_bands': [],
 'cloud_cover': 'CLOUD_COVER',
 'ee_collection': <ee.imagecollection.ImageCollection at 0x7f39f127b8d0>,
 'end_date': '2019-05-07',
 'id': 'LANDSAT/LC08/C01/T1_SR',
 'indices': {'evi': <function geetools.collection.Collection.evi(self, image, name='evi', renamed=False)>,
  'nbr

## get a band by id

In [8]:
l8sr.getBand('atmos_opacity')

In [9]:
l8sr.getBand('B1').name

'aerosol'

In [10]:
l8sr.getBand('red', 'name').id

'B4'

## Get one image out of the collection

In [11]:
l8sr_i = ee.Image(l8sr.collection.first())

In [14]:
print_values(l8sr_i)

{'B7': 3120, 'B10': 2639, 'pixel_qa': 480, 'B2': 8329, 'B5': 7478, 'B11': 2642, 'B6': 3754, 'B3': 8054, 'radsat_qa': 0, 'B4': 8031, 'sr_aerosol': 8, 'B1': 8394}


# Rename bands

In [15]:
l8sr_i.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

In [16]:
l8sr_renamed = l8sr.rename(l8sr_i, 'all')

In [17]:
l8sr_renamed.bandNames().getInfo()

['aerosol',
 'blue',
 'green',
 'red',
 'nir',
 'swir',
 'swir2',
 'thermal',
 'thermal2',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

## Add NDVI

In [18]:
l8sr_i_ndvi = l8sr.ndvi(l8sr_i)

In [19]:
print_values(l8sr_i_ndvi)

{'ndvi': -0.03156969811083914}


## Get Common Bands

In [20]:
l8toa = collection.Landsat8TOA()

In [21]:
for band in l8toa.bands:
    print(band.id, band.name)

B1 aerosol
B2 blue
B3 green
B4 red
B5 nir
B6 swir
B7 swir2
B8 pan
B9 cirrus
B10 thermal
B11 thermal2
BQA bqa


In [23]:
collection.getCommonBands(l8sr, l8toa, reference='all', match='name')

['green',
 'thermal',
 'swir',
 'nir',
 'aerosol',
 'thermal2',
 'red',
 'blue',
 'swir2']

## Get an Image

In [24]:
l8toa_i = ee.Image(l8toa.collection.first())

## Re-scale to match Landsat 8 TOA collection

For rescaling uses `min` and `max` information for the band that match in each collection

In [25]:
# SR
l8sr.ranges('all', False)

{'B1': {'max': 10000, 'min': 0},
 'B10': {'max': 10000, 'min': 0},
 'B11': {'max': 10000, 'min': 0},
 'B2': {'max': 10000, 'min': 0},
 'B3': {'max': 10000, 'min': 0},
 'B4': {'max': 10000, 'min': 0},
 'B5': {'max': 10000, 'min': 0},
 'B6': {'max': 10000, 'min': 0},
 'B7': {'max': 10000, 'min': 0},
 'pixel_qa': {'max': None, 'min': None},
 'radsat_qa': {'max': None, 'min': None},
 'sr_aerosol': {'max': None, 'min': None}}

In [26]:
# TOA
l8toa.ranges()

{'B1': {'max': 1, 'min': 0},
 'B10': {'max': 1000, 'min': 0},
 'B11': {'max': 1000, 'min': 0},
 'B2': {'max': 1, 'min': 0},
 'B3': {'max': 1, 'min': 0},
 'B4': {'max': 1, 'min': 0},
 'B5': {'max': 1, 'min': 0},
 'B6': {'max': 1, 'min': 0},
 'B7': {'max': 1, 'min': 0},
 'B8': {'max': 1, 'min': 0},
 'B9': {'max': 1, 'min': 0},
 'BQA': {'max': None, 'min': None}}

In [27]:
for band in l8sr.bands:
    print(band.name, band.id, band.precision)

aerosol B1 int16
blue B2 int16
green B3 int16
red B4 int16
nir B5 int16
swir B6 int16
swir2 B7 int16
thermal B10 int16
thermal2 B11 int16
sr_aerosol sr_aerosol uint8
pixel_qa pixel_qa uint16
radsat_qa radsat_qa uint16


In [28]:
# l8_i_scaled_toa = l8sr.rescale_all(l8sr_i, 8, 'TOA', drop=True)
l8_toa_scaled_sr = collection.rescale(l8toa_i, l8toa, l8sr, drop=False)
print_values(l8_toa_scaled_sr)

{'B7': 3132, 'B10': 2638, 'B9': 0.003796438919380307, 'B2': 7998, 'B5': 7538, 'B8': 0.7274518013000488, 'B11': 2642, 'B6': 3911, 'B3': 7182, 'B4': 7489, 'BQA': 2800, 'B1': 8138}


In [29]:
l8_i_scaled_toa = collection.rescale(l8toa_i, l8toa, l8sr, drop=True)
print_values(l8_i_scaled_toa)

{'B11': 2642, 'B6': 3911, 'B3': 7182, 'B10': 2638, 'B4': 7489, 'B5': 7538, 'B2': 7998, 'B7': 3132, 'B1': 8138}


In [30]:
l8_renamed_scaled_toa = collection.rescale(l8sr_renamed, l8sr, l8toa, renamed=True, drop=True)
print_values(l8_renamed_scaled_toa)

{'nir': 0.7477999925613403, 'thermal': 263.8999938964844, 'aerosol': 0.8393999934196472, 'thermal2': 264.20001220703125, 'red': 0.8030999898910522, 'swir2': 0.31200000643730164, 'green': 0.805400013923645, 'blue': 0.8328999876976013, 'swir': 0.37540000677108765}


In [31]:
l8_renamed_scaled_toa = collection.rescale(l8sr_renamed, l8sr, l8toa, renamed=True, drop=False)
print_values(l8_renamed_scaled_toa)

{'pixel_qa': 480, 'sr_aerosol': 8, 'green': 0.805400013923645, 'thermal': 263.8999938964844, 'swir': 0.37540000677108765, 'nir': 0.7477999925613403, 'radsat_qa': 0, 'aerosol': 0.8393999934196472, 'thermal2': 264.20001220703125, 'red': 0.8030999898910522, 'blue': 0.8328999876976013, 'swir2': 0.31200000643730164}


## Re-scale a collection

In [32]:
col = l8sr.collection.limit(5)

In [33]:
col_renamed = col.map(lambda img: l8sr.rename(img))

In [34]:
col_reescaled = col_renamed.map(lambda img: collection.rescale(img, l8sr, l8toa, drop=False, renamed=True))

In [35]:
tools.imagecollection.getValues(col_reescaled, ee.Image(col.first()).geometry().centroid(), 30, side='client')

{'LC08_001004_20140524': {'aerosol': 0.8393999934196472,
  'blue': 0.8328999876976013,
  'green': 0.805400013923645,
  'nir': 0.7477999925613403,
  'pixel_qa': 480.0,
  'radsat_qa': 0.0,
  'red': 0.8030999898910522,
  'sr_aerosol': 8.0,
  'swir': 0.37540000677108765,
  'swir2': 0.31200000643730164,
  'thermal': 263.8999938964844,
  'thermal2': 264.20001220703125},
 'LC08_001004_20140609': {'aerosol': 0.5275999903678894,
  'blue': 0.5042999982833862,
  'green': 0.4708999991416931,
  'nir': 0.4036000072956085,
  'pixel_qa': 480.0,
  'radsat_qa': 0.0,
  'red': 0.45190000534057617,
  'sr_aerosol': 8.0,
  'swir': 0.08799999952316284,
  'swir2': 0.09019999951124191,
  'thermal': 260.79998779296875,
  'thermal2': 260.8999938964844},
 'LC08_001004_20140625': {'aerosol': 0.42309999465942383,
  'blue': 0.4203000068664551,
  'green': 0.41110000014305115,
  'nir': 0.41990000009536743,
  'pixel_qa': 480.0,
  'radsat_qa': 0.0,
  'red': 0.4133000075817108,
  'sr_aerosol': 8.0,
  'swir': 0.39320001006

# From Id

In [36]:
l5toa = collection.Landsat.fromId('LANDSAT/LT05/C01/T1_TOA')

In [37]:
collection.info(l5toa)

{'algorithms': {'brdf': <bound method Landsat.brdf of <geetools.collection.landsat.Landsat object at 0x7f39f11d5c88>>,
  'harmonize': <bound method Landsat.harmonize of <geetools.collection.landsat.Landsat object at 0x7f39f11d5c88>>,
  'rescale': <bound method Landsat.rescaleAll of <geetools.collection.landsat.Landsat object at 0x7f39f11d5c88>>},
 'band_names': ['blue',
  'green',
  'red',
  'nir',
  'swir',
  'thermal',
  'swir2',
  'bqa'],
 'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'BQA'],
 'bit_bands': ['BQA'],
 'classification_bands': [],
 'cloud_cover': 'CLOUD_COVER',
 'ee_collection': <ee.imagecollection.ImageCollection at 0x7f39f11d5b70>,
 'end_date': '2012-05-05',
 'id': 'LANDSAT/LT05/C01/T1_TOA',
 'indices': {'evi': <function geetools.collection.Collection.evi(self, image, name='evi', renamed=False)>,
  'nbr': <function geetools.collection.Collection.nbr(self, image, name='nbr', renamed=False)>,
  'ndvi': <function geetools.collection.Collection.ndvi(self, image, nam

In [39]:
l5toa.getBand('B3').name

'red'

## Visualize

In [4]:
Map = Map()

In [5]:
Map.show()

Map(basemap={'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a…

In [6]:
rect = ee.Geometry.Rectangle([-71.57, -40.81, -71.43, -40.72])

In [7]:
Map.addLayer(rect)

In [11]:
p = ee.Geometry.Point([-71.63085937500001, -40.76448635693034])

In [12]:
l8sri = l8sr.collection.filterBounds(p).filterMetadata(l8sr.cloud_cover, 'less_than', 30).first()

In [13]:
vis = l8sr.visualization('NSR')

In [14]:
vis

{'bands': ['B5', 'B6', 'B4'],
 'max': [5000.0, 5000.0, 5000.0],
 'min': [0, 0, 0]}

In [48]:
Map.addLayer(l8sri, vis, 'L8 SR NSR')

In [49]:
Map.centerObject(l8sri)

In [16]:
proxy = l8sr.proxyImage()

In [17]:
eprint(proxy)

## Mask

In [55]:
l8sr.bitOptions()

{'pixel_qa': ['cloud',
  'water',
  'high_confidence_cirrus',
  'clear',
  'high_confidence_cloud',
  'snow',
  'shadow'],
 'radsat_qa': ['B1_saturated',
  'B2_saturated',
  'B3_saturated',
  'B4_saturated',
  'B5_saturated',
  'B6_saturated',
  'B7_saturated',
  'B9_saturated',
  'B10_saturated',
  'B11_saturated'],
 'sr_aerosol': ['climatology', 'low', 'medium', 'high', 'water']}

In [57]:
masked = l8sr.applyMask(l8sri, 'pixel_qa', ['cloud', 'snow', 'shadow'])

In [58]:
Map.addLayer(masked, vis, 'Masked L8 SR (pixel-qa)')